# Part 2. Model Training & Evaluation - RNN

## Prepare embedding matrix and metadata

In [19]:
import json
from pathlib import Path

import numpy as np

embedding_path = Path("models/embedding_matrix.npy")
index_from_word_path = Path("models/index_from_word.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

## Prepare dataset

In [ ]:
from utils.text import tokenize
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = tokenize(dataset["train"])
val_dataset = tokenize(dataset["validation"])
test_dataset = tokenize(dataset["test"])

In [ ]:
from utils.text import token_to_index

train_dataset = token_to_index(dataset=train_dataset, index_from_word=index_from_word)
val_dataset = token_to_index(dataset=val_dataset, index_from_word=index_from_word)
test_dataset = token_to_index(dataset=test_dataset, index_from_word=index_from_word)

In [22]:
train_dataset = train_dataset.select_columns(["label", "original_len", "indexes"])
val_dataset = val_dataset.select_columns(["label", "original_len", "indexes"])
test_dataset = test_dataset.select_columns(["label", "original_len", "indexes"])

In [23]:
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

In [24]:
train_dataset

Dataset({
    features: ['label', 'original_len', 'indexes'],
    num_rows: 8530
})

## Train RNN Model

In [164]:
SEARCH_SPACE = {
    "batch_size": [32, 64, 128, 256, 512, 1024, 2048],
    "learning_rate": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    "optimizer_name": ["SGD", "Adagrad", "RMSprop", "Adam"],
    # RNN Model Parameters
    "hidden_dim": [256, 128, 64, 32],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["last", "average", "max"],
}

In [ ]:
from utils.train import train_rnn_model_with_parameters

for hidden_dim in SEARCH_SPACE["hidden_dim"]:
    for num_layers in SEARCH_SPACE["num_layers"]:
        for optimizer_name in SEARCH_SPACE["optimizer_name"]:
            for batch_size in SEARCH_SPACE["batch_size"]:
                for learning_rate in SEARCH_SPACE["learning_rate"]:
                    for sentence_representation_type in SEARCH_SPACE["sentence_representation_type"]:
                        log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type}  ----------"
                        print(log_message)
                        train_rnn_model_with_parameters(
                            embedding_matrix=embedding_matrix,
                            train_dataset=train_dataset,
                            val_dataset=val_dataset,
                            batch_size=batch_size,
                            learning_rate=learning_rate,
                            optimizer_name=optimizer_name,
                            hidden_dim=hidden_dim,
                            num_layers=num_layers,
                            sentence_representation_type=sentence_representation_type,
                            show_progress=True,
                        )

## Model Configurations Comparison

In [137]:
from utils.analytics import load_tensorboard_logs

train_results_df = load_tensorboard_logs(log_dir="tb_logs")
train_results_df = train_results_df.sort_values(
    by=["val_acc"], ascending=False
).reset_index(drop=True)
train_results_df.head(10)

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,num_layers,sentence_representation_type,steps,train_loss,val_loss,filename
0,0.757974,0.807068,2048,256,0.00001,Adam,1,last,1449.0,0.454111,0.491301,events.out.tfevents.1729879439.yuriarch.4562.53
1,0.757172,0.802853,1024,256,0.00010,Adam,1,last,349.0,0.427634,0.488983,events.out.tfevents.1729880467.yuriarch.4562.55
2,0.754683,0.778416,1024,128,0.00010,Adam,1,last,549.0,0.464212,0.483763,events.out.tfevents.1729882559.yuriarch.4562.61
3,0.749685,0.783276,1024,128,0.00010,Adam,2,average,1099.0,0.458963,0.499223,events.out.tfevents.1729883053.yuriarch.4562.64
4,0.747005,0.819788,1024,128,0.00010,Adam,4,last,449.0,0.398053,0.492108,events.out.tfevents.1729883387.yuriarch.4562.65
5,0.746843,0.819348,1024,256,0.00010,Adam,1,last,299.0,0.412483,0.490003,events.out.tfevents.1729878898.yuriarch.4562.49
6,0.746612,0.814443,1024,256,0.00001,Adam,1,last,1349.0,0.431975,0.492996,events.out.tfevents.1729878529.yuriarch.4562.48
7,0.742492,0.796431,1024,256,0.00010,Adam,2,last,249.0,0.460812,0.496632,events.out.tfevents.1729881152.yuriarch.4562.57
8,0.739449,0.791667,1024,128,0.00010,Adam,2,last,449.0,0.467429,0.507560,events.out.tfevents.1729882970.yuriarch.4562.63
9,0.739327,0.815672,1024,128,0.00010,Adam,1,average,1199.0,0.454116,0.509695,events.out.tfevents.1729882639.yuriarch.4562.62


# (a) Final Configuration of best model

In [138]:
best_rnn_model_configuration = train_results_df.head(1)
best_rnn_model_configuration

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,num_layers,sentence_representation_type,steps,train_loss,val_loss,filename
0,0.757974,0.807068,2048,256,0.00001,Adam,1,last,1449.0,0.454111,0.491301,events.out.tfevents.1729879439.yuriarch.4562.53


In [152]:
from pathlib import Path


best_rnn_model_filename = best_rnn_model_configuration["filename"].item()
matched_files = list(Path().rglob(best_rnn_model_filename))

if not matched_files:
    print("Model checkpoint not found!")
else:
    checkpoint_dir = matched_files[0].parent / "checkpoints"
    checkpoint_files = (
        list(checkpoint_dir.glob("*.ckpt")) if checkpoint_dir.exists() else []
    )

    if not checkpoint_files:
        print("No checkpoint files found in the checkpoint directory!")
    else:
        best_checkpoint = checkpoint_files[0] 
        best_rnn_model = RNNClassifier.load_from_checkpoint(best_checkpoint)
        print(best_rnn_model)

RNNClassifier(
  (model): RNN(
    (embedding): Embedding(14687, 300)
    (rnn): RNN(300, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=128, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=128, out_features=2, bias=True)
  )
  (metric): MulticlassAccuracy()
)


/home/yuri/miniconda3/envs/habitat/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'rnn_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['rnn_model'])`.


# (b) Accuracy on Testset

In [157]:
test_dataloader = DataLoader(test_dataset, shuffle=True)

trainer = L.Trainer(accelerator="cpu")
trainer.test(best_rnn_model, test_dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/home/yuri/miniconda3/envs/habitat/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7636022567749023     │
│         test_loss         │    0.4737122356891632     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4737122356891632, 'test_acc': 0.7636022567749023}]

# (c) Strategies to derive final sentence representation

1. Last State: Use last hidden state as sentence representation
2. Max Pooling
3. Average Pooling

In [158]:
best_rnn_model_configuration

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,num_layers,sentence_representation_type,steps,train_loss,val_loss,filename
0,0.757974,0.807068,2048,256,0.00001,Adam,1,last,1449.0,0.454111,0.491301,events.out.tfevents.1729879439.yuriarch.4562.53


In [ ]:
# Best model configs
batch_size = int(best_rnn_model_configuration['batch_size'].values[0])
learning_rate = float(best_rnn_model_configuration['learning_rate'].values[0])
optimizer_name = str(best_rnn_model_configuration['optimizer_name'].values[0])
hidden_dim = int(best_rnn_model_configuration['hidden_dim'].values[0])
num_layers = int(best_rnn_model_configuration['num_layers'].values[0])

for sentence_representation_type in SEARCH_SPACE["sentence_representation_type"]:
    print(f"---------- sentence_representation_type: {sentence_representation_type}; batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers} ----------")
    train_rnn_model_with_parameters(
        batch_size=batch_size,
        learning_rate=learning_rate,
        optimizer_name=optimizer_name,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        sentence_representation_type=sentence_representation_type,
        show_progress=True,
        log_dir="test_sentence_representation",
    )

In [168]:
from utils.analytics import load_tensorboard_logs

test_results_df = load_tensorboard_logs("tb_logs/test_sentence_representation")
test_results_df = test_results_df.sort_values(
    by=["val_acc"], ascending=False
).reset_index(drop=True)
test_results_df

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,num_layers,sentence_representation_type,epoch,train_loss,val_loss,filename
0,0.761726,0.807068,2048,256,0.00001,Adam,1,last,298.0,0.454111,0.491746,events.out.tfevents.1729891732.yuriarch.4562.70
1,0.754221,0.773876,2048,256,0.00001,Adam,1,max,267.0,0.471278,0.505773,events.out.tfevents.1729909635.yuriarch.4562.72
2,0.741088,0.735488,2048,256,0.00001,Adam,1,average,507.0,0.533019,0.532041,events.out.tfevents.1729892650.yuriarch.4562.71
